# Update ontology sources

In [ ]:
!lamin connect laminlabs/bionty-assets

In [ ]:
import bionty as bt
import lamindb as ln
from bionty.core._source import register_source_in_bionty_assets
from lamin_utils import logger

ln.settings.verbosity = "hint"

ln.track("7extigZj6QNG")

All entities that are not listed in the following matrix must be curated manually as they require intervention.
Consult https://bionty-assets-gczz.netlify.app/ingest/ for guidance.

In [ ]:
def process_ontology_entities(entity_configs):
    """Process ontology entities with their source configurations.

    Args:
        entity_configs (list): List of tuples containing (entity, source_name, organism, *version)
    """
    from bionty.base._ontology_url import get_ontology_url

    total_configs = len(entity_configs)
    success_count = 0
    error_count = 0

    for i, config in enumerate(entity_configs, 1):
        entity, source_name, organism, *version = config
        config_id = f"{entity}_{source_name}_{organism}"

        logger.info(f"[{i}/{total_configs}] Processing {config_id}")

        try:
            logger.debug(
                f"Getting ontology URL for {source_name}, requested version: {version[0] if version else 'latest'}"
            )
            *_, version_to_use = get_ontology_url(
                prefix=source_name, version=version[0] if version else None
            )
            logger.info(
                f"Processing... {entity:<20} {source_name:<10} {version_to_use:<12} {organism}"
            )

            new_df = getattr(bt.base, entity)(
                source=source_name, version=version_to_use
            ).df()

            logger.debug(
                f"Checking for existing source: entity=bionty.{entity}, name={source_name}, organism={organism}"
            )
            try:
                currently_used_source = bt.Source.filter(
                    entity=f"bionty.{entity}",
                    name=source_name,
                    organism=organism,
                    currently_used=True,
                ).one_or_none()

                if currently_used_source:
                    current_version_df = getattr(bt.base, entity)(
                        source=currently_used_source
                    ).df()
                    n_old = new_df.shape[0]
                    n_new = current_version_df.shape[0]
                    if n_old < n_new:
                        raise ValueError(
                            f"The new version has less rows: {n_new} < {n_old}"
                        )
            except ValueError as e:
                if "No source url is available" in str(e):
                    pass  # This occurs during testing in local instances
                else:
                    raise

            logger.debug(f"Adding source record for {entity}")
            source_rec = getattr(bt, entity).add_source(
                source=source_name, version=version_to_use
            )

            logger.info(f"Registering assets for {config_id}...")
            try:
                register_source_in_bionty_assets(
                    f"bionty/base/_dynamic/df_{organism}__{source_name}__{version_to_use}__{entity}.parquet",
                    source=source_rec,
                    is_dataframe=True,
                )
                register_source_in_bionty_assets(
                    f"bionty/base/_dynamic/ontology_{organism}__{source_name}__{version_to_use}__{entity}",
                    source=source_rec,
                    is_dataframe=False,
                )
                logger.info(
                    f"Registered a new version {version_to_use} of {entity}. Ensure that it is the latest version in the `source.yaml` file."
                )
                success_count += 1

            except ValueError as e:
                if "artifact already exists" in str(e):
                    logger.warning(
                        f"Entity {entity} using source {source_name} of version {version_to_use} and organism {organism} is already registered. Skipping..."
                    )
                    success_count += 1
                else:
                    raise
            except FileNotFoundError:
                logger.warning(
                    f"Entity {entity} using source {source_name} of version {version_to_use} and organism {organism} file cannot be found. "
                    "This can happen if the ontology was previously registered and the pronto ontology file did not get recreated. Skipping..."
                )
                success_count += 1

        except Exception as e:
            error_count += 1
            logger.error(
                f"[{i}/{total_configs}] {config_id} failed: {type(e).__name__}: {str(e)}"
            )
            continue

    logger.info(
        f"Total: {total_configs}, success: {success_count}, failed: {error_count}"
    )

In [ ]:
configs = [
    ("Disease", "mondo", "all"),
    ("CellType", "cl", "all"),
    ("Organism", "ncbitaxon", "all"),
    ("Tissue", "uberon", "all"),
    ("Disease", "doid", "human"),
    ("ExperimentalFactor", "efo", "all"),
    ("Phenotype", "pato", "all"),
    ("Phenotype", "hp", "human"),
    ("Pathway", "go", "all"),
    # ("Pathway", "pw", "all"), Currently leads to a URL error - upstream issue
    ("DevelopmentalStage", "hsapdv", "human"),
    ("DevelopmentalStage", "mmusdv", "mouse"),
    ("Ethnicity", "hancestro", "human"),
    # ("Drug", "dron", "all"), Not a Bionty entity (yet)
]

process_ontology_entities(configs)

In [ ]:
ln.finish()